In [1]:
##%% IMPORTING LIBRARIES

import os
import glob
import shutil
import random
import pandas as pd
from PIL import Image
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import zipfile



In [2]:
##%% IMPORTING DATA

def importing_data(path):
    sample = []
    for filename in glob.glob(path):
        #img = Image.open(filename,'r')
        #IMG = np.array(img)
        sample.append(filename)
    return sample

path1 = 'archive/no/*.jpg'
path2 = 'archive/yes/*.jpg'
path3 = 'archive/pred/*.jpg'
path4 = 'archive/xyz/*.jpg'
train_n = importing_data(path1)
train_y = importing_data(path2)
test = importing_data(path3)
xyz_test = importing_data(path4)

##%% CREATION OF DATASETS

df_train_n = pd.DataFrame({'image':train_n, 'label': 'Healthy'})
df_train_y = pd.DataFrame({'image':train_y, 'label': 'Affected'})
df_test = pd.DataFrame({'image':test})
xyz_test=pd.DataFrame({'image':xyz_test})
train_data = pd.concat([df_train_n, df_train_y])
train_data.head()

,image,label
0,archive/no\no0.jpg,Healthy
1,archive/no\no1.jpg,Healthy
2,archive/no\no10.jpg,Healthy
3,archive/no\no100.jpg,Healthy
4,archive/no\no1000.jpg,Healthy


In [3]:
##%% TRAIN-VALIDATION SPLIT (90% TRAIN - 10% VALIDATION)

from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(train_data,
                                  test_size = 0.1,
                                  shuffle = True,
                                  random_state = 42)

<a id="subsection-one"></a>
### CREATING THE CNN MODEL

In [4]:
##%% CREATING THE CNN MODEL 

import keras
from keras.metrics import AUC, Recall, Precision
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D , MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import RMSprop

def build_model():
    
    '''Sequential Model creation'''
    Cnn = Sequential()
    
    Cnn.add(Conv2D(64,(5,5), activation = 'relu', padding = 'same',
                   strides=(2,2), input_shape = [224,224,1]))
    Cnn.add(MaxPooling2D(2))
    Cnn.add(Conv2D(128,(5,5), activation = 'relu', padding = 'same', strides=(2,2)))
    Cnn.add(Conv2D(128,(5,5), activation = 'relu', padding = 'same', strides=(2,2)))
    Cnn.add(Conv2D(256,(5,5), activation = 'relu', padding = 'same', strides=(2,2)))
    Cnn.add(MaxPooling2D(2))
    #Cnn.add(GlobalAveragePooling2D())
    Cnn.add(Flatten())
    Cnn.add(Dense(64, activation = 'relu'))
    Cnn.add(Dropout(0.4))
    Cnn.add(Dense(32, activation = 'relu'))
    Cnn.add(Dropout(0.4))
    Cnn.add(Dense(2, activation = 'softmax'))
    
    return Cnn

keras_model = build_model()
keras_model.summary()

C:\Users\DELL\miniconda3\envs\detect\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 112, 112, 64)        │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 28, 28, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 14, 14, 128)         │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 7, 7, 256)           │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         147,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,585,442 (6.05 MB)

 Trainable params: 1,585,442 (6.05 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
##%% FITTING THE MODEL

def Model_fit(train_data, val_data):
    
    keras_model = None
    
    keras_model = build_model()
    
    '''Compiling the model'''
    
    keras_model.compile(optimizer = RMSprop(learning_rate = 1e-4),
                        loss='sparse_categorical_crossentropy',
                        metrics =['acc'])
    
    es = EarlyStopping(monitor='val_loss', mode='min',
                       patience=2,
                       restore_best_weights=True,
                       verbose=1)
                       
    
    checkpoint_cb = ModelCheckpoint("Brain_model_best.keras",
                                    save_best_only=True)
    
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                                  factor = 0.2,
                                  patience = 3,
                                  min_lr = 1e-5,
                                  mode = 'min',
                                  verbose=1)
                                  
    
    history = keras_model.fit(train_data,
                              validation_data = val_data,
                              epochs= 1,
                              batch_size = 10,
                              callbacks=[es, checkpoint_cb, reduce_lr])
                              
    
      
    return history

In [7]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

k_fold = 3
IMG_SIZE = 224
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 2

def CV_training(train_data, val_data):
    
    cv_histories = []
    
    for i in range(0,k_fold):
    
        datagen = ImageDataGenerator(rescale = 1./255)
    
        train_set = datagen.flow_from_dataframe(train_data,
                                                directory = 'archive/RCNN/TRAIN/*.jpg',
                                                x_col = 'image',
                                                y_col = 'label',
                                                target_size = size,
                                                color_mode = 'grayscale',
                                                class_mode = 'sparse',
                                                batch_size = 10,
                                                shuffle = True,
                                                interpolation = 'bilinear')
        
        val_set = datagen.flow_from_dataframe(val_data,
                                              directory = 'archive/RCNN/VAL/*.jpg',
                                              x_col = 'image',
                                              y_col = 'label',
                                              target_size = size,
                                              color_mode = 'grayscale',
                                              class_mode = 'sparse',
                                              batch_size = 10,
                                              shuffle = True,
                                              interpolation = 'bilinear')
        print("Training on Fold: ",i+1)
    
        cv_histories.append(Model_fit(train_set, val_set))
    
    return cv_histories

cv_results = CV_training(X_train,X_val)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\DELL\miniconda3\envs\detect\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 2700 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(
C:\Users\DELL\miniconda3\envs\detect\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 300 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(
C:\Users\DELL\miniconda3\envs\detect\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Training on Fold:  1


ValueError: Must provide at least one structure

In [ ]:
!pwd

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras.models import Sequential

k_fold = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)
n_CLASS = 2

def Model_fit(train_data, val_data):
    keras_model = Sequential()
    # Define your model architecture here
    # Example architecture:
    # keras_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
    # keras_model.add(MaxPooling2D(pool_size=(2, 2)))
    # keras_model.add(Flatten())
    # keras_model.add(Dense(128, activation='relu'))
    # keras_model.add(Dense(n_CLASS, activation='softmax'))

    keras_model.compile(optimizer=RMSprop(learning_rate=1e-4),
                        loss='sparse_categorical_crossentropy',
                        metrics=['acc'])

    es = EarlyStopping(monitor='val_loss', mode='min',
                       patience=2,
                       restore_best_weights=True,
                       verbose=1)

    checkpoint_cb = ModelCheckpoint("Brain_model_best.keras",
                                    save_best_only=True)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2,
                                  patience=3,
                                  min_lr=1e-5,
                                  mode='min',
                                  verbose=1)

    history = keras_model.fit(train_data,
                              validation_data=val_data,
                              epochs=50,
                              batch_size=10,
                              callbacks=[es, checkpoint_cb, reduce_lr])

    return history

def CV_training(train_data, val_data):
    cv_histories = []

    for i in range(k_fold):
        datagen = ImageDataGenerator(rescale=1./255)

        train_set = datagen.flow_from_dataframe(dataframe=train_data,
                                                directory='archive/RCNN/TRAIN',
                                                x_col='image',
                                                y_col='label',
                                                target_size=size,
                                                color_mode='grayscale',
                                                class_mode='sparse',
                                                batch_size=10,
                                                shuffle=True,
                                                interpolation='bilinear')

        val_set = datagen.flow_from_dataframe(dataframe=val_data,
                                              directory='archive/RCNN/VAL',
                                              x_col='image',
                                              y_col='label',
                                              target_size=size,
                                              color_mode='grayscale',
                                              class_mode='sparse',
                                              batch_size=10,
                                              shuffle=True,
                                              interpolation='bilinear')

        print("Training on Fold:", i + 1)
        # Add error checking for empty data
        if train_set.samples == 0 or val_set.samples == 0:
            print("Error: Empty data in fold", i + 1)
            continue

        cv_histories.append(Model_fit(train_set, val_set))

    return cv_histories

# Check if X_train and X_val have data
if X_train.empty or X_val.empty:
    print("Error: Empty data in X_train or X_val")
else:
    cv_results = CV_training(X_train, X_val)


In [ ]:
import os

def count_images_in_folder(folder_path):
    # Check if the folder path exists
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return

    # Initialize a counter for the number of images
    image_count = 0

    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image based on its extension
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_count += 1

    return image_count

# Specify the folder path you want to count images in
folder_path = 'archive/RCNN/TRAIN'

# Call the function to count images in the folder
num_images = count_images_in_folder(folder_path)

if num_images is not None:
    print(f"Found {num_images} images in the folder '{folder_path}'.")


In [ ]:
import cv2
import numpy as np

k_fold = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)
n_CLASS = 2

def load_images_and_labels(dataframe, directory):
    images = []
    labels = []
    for index, row in dataframe.iterrows():
        image_path = os.path.join(directory, row['image'])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, size, interpolation=cv2.INTER_LINEAR)
        images.append(image)
        labels.append(row['label'])
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def CV_training(train_data, val_data):
    cv_histories = []
    
    for i in range(0, k_fold):
        train_images, train_labels = load_images_and_labels(train_data, 'archive/RCNN/TRAIN/')
        val_images, val_labels = load_images_and_labels(val_data, 'archive/RCNN/VAL/')
        
        train_images = train_images / 255.0
        val_images = val_images / 255.0
        
        print("Training on Fold: ", i + 1)
        cv_histories.append(Model_fit(train_images, train_labels, val_images, val_labels))
    
    return cv_histories

cv_results = CV_training(X_train, X_val)


In [ ]:
import os

def check_path_validity(file_path):
    if os.path.exists(file_path):
        print(f"Path '{file_path}' exists.")
        if os.path.isfile(file_path):
            print(f"'{file_path}' is a valid file path.")
        else:
            print(f"'{file_path}' is not a file path.")
    else:
        print(f"Path '{file_path}' does not exist.")

# Example usage
file_path = 'archive/RCNN/TRAIN'
check_path_validity(file_path)


In [ ]:
##%% CHEKING THE CROSS VALIDATION METRICS

def acc_results(results):
    i = 0
    for fold in cv_results:
        print('Val_Acc Folder '+ str(i) + ' =', max(fold.history['val_acc']))
        i += 1
        
acc_results(cv_results)

In [ ]:
##%% LOOKING AT THE ACCURACY-LOSS PLOTS FOR EACH FOLD

def Acc_Loss_Plot(results):
    
    for fold in results:
        
        acc = fold.history['acc']
        val_acc = fold.history['val_acc']
        loss = fold.history['loss']
        val_loss = fold.history['val_loss']
    
        fig, (ax1, ax2) = plt.subplots(1,2, figsize= (10,5))
        fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

        ax1.plot(range(1, len(acc) + 1), acc)
        ax1.plot(range(1, len(val_acc) + 1), val_acc)
        ax1.set_title('History of Accuracy')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Accuracy')
        ax1.legend(['training', 'validation'])


        ax2.plot(range(1, len(loss) + 1), loss)
        ax2.plot(range(1, len(val_loss) + 1), val_loss)
        ax2.set_title('History of Loss')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend(['training', 'validation'])
        plt.show()
    
Acc_Loss_Plot(cv_results)

In [ ]:
##%% LOADING THE MODEL

import keras

keras_model = keras.models.load_model('Brain_model_best.h5')
keras_model.compile(optimizer = RMSprop(learning_rate = 1e-4),
                    loss='sparse_categorical_crossentropy', metrics =[ 'acc'])

# Predictions on the test set

datagen = ImageDataGenerator(rescale = 1./255)

test_set = datagen.flow_from_dataframe(df_test,
                                       directory = 'C:/vansh/programming/project/python/archive/Br35H-Mask-RCNN/TEST/*.jpg',
                                       x_col = 'image',
                                       y_col = None,
                                       target_size = size,
                                       color_mode = 'grayscale',
                                       class_mode = None,
                                       batch_size = 10,
                                       shuffle = False,
                                       interpolation = 'bilinear')

predictions = keras_model.predict(test_set)
predictions = predictions.argmax(axis=-1)
print("Where 0 = 'Affected'")
print("Where 1 = 'Healthy'")
print(predictions)


In [ ]:
pred = []
[pred.append('Healthy') if i == 1 else pred.append('Affected') for i in predictions]
print(pred)

In [ ]:
##%% OBTAINING PREDICTIONS OF THE FIRST BATCH
    
images10 = [test_set[0][0],test_set[0][1],test_set[0][2],test_set[0][3],test_set[0][4],
            test_set[0][5],test_set[0][6],test_set[0][7],test_set[0][8],test_set[0][9]]
            
prediction10 = pred[0:9]
final_pred = zip(images10,prediction10)

In [ ]:
def pre_visualization(data, predictions):
    
    for image,pred in final_pred:
        plt.imshow(image.reshape(224,224), cmap = 'gray')
        plt.title("Model's Prediction: " + str(pred))
        plt.show()
        
pre_visualization(images10,prediction10)

In [ ]:
keras_model = keras.models.load_model('Brain_model_best.h5')
keras_model.compile(optimizer = RMSprop(learning_rate = 1e-4),
                    loss='sparse_categorical_crossentropy', metrics =[ 'acc'])

# Predictions on the test set

datagen1 = ImageDataGenerator(rescale = 1./255)

xyz_set = datagen1.flow_from_dataframe(xyz_test,
                                       directory = 'C:/vansh/programming/project/python/archive/Br35H-Mask-RCNN/xyz/y1.jpg',
                                       x_col = 'image',
                                       y_col = None,
                                       target_size = size,
                                       color_mode = 'grayscale',
                                       class_mode = None,
                                       batch_size = 10,
                                       shuffle = False,
                                       interpolation = 'bilinear')

predictions1 = keras_model.predict(xyz_set)
predictions1 = predictions1.argmax(axis=-1)
print("Where 0 = 'Affected'")
print("Where 1 = 'Healthy'")
print(predictions1)

In [ ]:
pred1 = []
[pred1.append('Healthy') if i == 1 else pred1.append('Affected') for i in predictions1]
print(pred1)

In [ ]:
images_xyz = [xyz_set[0][0]]
            
prediction_xyz = pred1[0:9]
final_xyz_pred = zip(images_xyz,prediction_xyz)

In [ ]:
def pre_visualization(data, predictions1):
    
    for image,pred in final_xyz_pred:
        plt.imshow(image.reshape(224,224), cmap = 'gray')
        plt.title("Model's Prediction: " + str(pred))
        plt.show()
        
pre_visualization(images_xyz,prediction_xyz)

<a id="section-two"></a>
### CAM - VISUALIZATION 

In [ ]:
##%% IMPORTING LIBRARIES

import numpy as np
import tensorflow as tf
import matplotlib.cm as cm
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython.display import Image

Code adapted from: https://keras.io/examples/vision/grad_cam/

In [ ]:
layer_names = [layer.name for layer in keras_model.layers]
layer_names

In [ ]:
##%% CREATING THE HEATMAP FROM THE LAYERS' ACTIVATIONS

img_size = (224, 224)
layer_names=[layer.name for layer in keras_model.layers]

last_conv_layer_name = 'conv2d_15'
classifier_layer_names = [
    'max_pooling2d_7',
    'flatten_3',
    'dense_9',
    'dropout_6',
    'dense_10',
    'dropout_7',
    'dense_11']

def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    
    #img_array = test_set[0][8]
    img_array = img_array.reshape(1,224,224,1)
    img_array = img_array[:1]
    
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1,2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap


In [ ]:
##%% DEFINING A FUNCTION TO DISPLAY THE HEATMAP ON THE REAL IMAGE

def display(heatmap, img):
    heatmap = np.uint8(255 * heatmap)
    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[1]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    
    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.005 + img.reshape(224,224,1)  #img_array.reshape(224,224,1)
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img

In [ ]:
##%% OBSERVING THE RAW IMAGE, THE HEATMAP AND THE SUPERIMPOSED IMAGES TOGETHER

for img in images10:
    heatmap = make_gradcam_heatmap(img,
                                   keras_model,
                                   last_conv_layer_name,
                                   classifier_layer_names)
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize= (10,5))
    
    ax1.imshow(img.reshape(224,224), cmap = 'gray')
    ax1.set_title('Raw MRI image')
    ax2.matshow(heatmap)
    ax3.imshow(display(heatmap,img))
    ax3.set_title('Superimposed Activation Heatmap')

In [ ]:

for img in images_xyz:
    heatmap = make_gradcam_heatmap(img,
                                   keras_model,
                                   last_conv_layer_name,
                                   classifier_layer_names)
    fig, (ax4, ax5, ax6) = plt.subplots(1,3, figsize= (10,5))
    
    ax4.imshow(img.reshape(224,224), cmap = 'gray')
    ax4.set_title('Raw MRI image')
    ax5.matshow(heatmap)
    ax6.imshow(display(heatmap,img))
    ax6.set_title('Superimposed Activation Heatmap')